# RFM nedir?

R harfi Recency’i,

F harfi Frequency’i

M harfi Monetary’i temsil etmektedir.

Bu üç metriğin hesaplanmasından sonra birleştirilmesiyle meydana gelen bir skordur. Müşterilerin mevcut durumunun analiz edilip, bu skorlara göre segmentlere ayrılmasına yardımcı olur.

* Recency: Müşterinin ne kadardır websitesinden/mağazadan hizmet aldığı, ne zamandır bize üye olduğu gibi bilgileri verir. Hesaplanması genellikle, bugünden son üyelik tarihi/son sipariş tarihinin çıkartılmasıyla elde edilir.

* Frequency: Müşterinin ne sıklıkla alışveriş yaptığını, ne sıklıkla siteye giriş yaptığını gösteren metriktir. Genellikle sipariş numarası/sipariş kodunun saydırılmasıyla sonuç verir.

* Monetary: Müşterinin harcamalarının toplamıdır. E-ticaret sitesine getirdiği ciro, aldığı hizmetler sonrası toplanan getiri olarak da tanımlanabilir. Ciro tanımı ne ise, müşteri bazında hayatı boyunca yapılan harcamalar toplanarak hesaplanır.


**Not : Bu proje Veri Bilimi Okulu Bootcamp için yazılmıştır. Desteklerinden dolayı M.Vahit Keskin'e ve Günce Daşçı'ya teşekkür ederim**

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

**Gerekli kütüphanelerin import edilmesinden sonra veri hakkında bilgi ediniyoruz.**

In [ ]:
df = pd.read_csv("../input/online-retail-ii-uci/online_retail_II.csv")

In [ ]:
df.info()

In [ ]:
'''
InvoiceDate  kolonu datetime olmadığından bunu datetime a dönüştürüyoruz. Bu sayede hesaplamamız kolay oluyor
'''
df['InvoiceDate'] =  pd.to_datetime(df['InvoiceDate'])
df.info()

In [ ]:
df.head()

In [ ]:
'''
Eşsiz ürün sayısını inceliyoruz
'''
df["Description"].nunique()

In [ ]:
'''
Eşsiz ürün listesinden ilk 5 ürünün toplam ne kadar veri setinde geçtiğine bakıyoruz.
'''
df["Description"].value_counts().head()

In [ ]:
'''
    Veri setimizde quintity kolonunda aykırı verileri inceliyoruz. Göründüğü gibi (-) değerler var. Bu da iadelerden kaynaklı. Eğer Invoice kolonundan içinde 'C' geçenleri kaldırırsak 
bu sorunu çözmüş olacağız.İçinde 'C' harfi geçenleri ilerki aşamalarda kaldırmış olacağız
'''

df.Quantity.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99])

In [ ]:
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head()

In [ ]:
df["Invoice"].nunique()

In [ ]:
'''
Verimize TotalPrice kısmını ekliyoruz. Bir faturada ne kadar harcadığını daha kolay bulabilmek için.
Ve ardından üst kısımlarda bahsettiğim iade edilen faturaları siliyoruz. RFM de bunlara ihtiyacımız olmucak.
'''
df["TotalPrice"] = df["Quantity"] * df["Price"]
df = df[~df["Invoice"].str.contains("C", na=False)]

In [ ]:
df.sort_values("TotalPrice", ascending=False).head()

In [ ]:
"""
Müşterilerin yaşadığı ilk 5 ülkeyi sns ile bar plot ile görüntülüyoruz
"""
country_plot = pd.DataFrame(df.Country.value_counts().head(5))
sns.barplot(country_plot.Country,country_plot.index)
plt.show()

In [ ]:
"""
Ülkelerde toplam ne kadar harcağının sayısını inceliyoruz. Burada UK nin açık ara en fazla olmasının nedeni üst kısımda gördüğümüz gibi müşteri sayısının açık ara önde olmasıdır.
"""
df.groupby("Country").agg({"TotalPrice": "sum"}).sort_values("TotalPrice", ascending=False).head()

In [ ]:
"""
Kolonlardaki null sayılarını grafik ile görüntülüyoruz. En fazla null sayısı CustumerID de. Bunları verilerimizden siliyoruz.
"""
val_null = pd.DataFrame(df.isnull().sum(),columns=['val_count'])
sns.barplot(val_null.val_count,val_null.index)
plt.show()

In [ ]:
df.dropna(inplace=True)
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

In [ ]:
"""
Tekrar info ya baktığımda verilerimizde null değer kalmamış ve bütün kolonlar istediğimiz değişken tipinde.
CustumerID nin object olması daha doğru olacaktır. Çünkü onla bir sayısal işlem yapmayacağız. Ama bu projede bu sıkıntı çokta bizi alakadar etmiyor.
"""
df.info()

# RFM Skor Hesaplanması

In [ ]:
"""
Burada en son ne zaman alışveriş yapıldığına bakıyoruz. Today_date olarak şimdiki zamanuı almalıyız ancak veri seti en son 2011 sonunda bittiğinden şimdiki zamanı alırsak tamamen yanlış analiz
yapmış oluruz. Bu yüzden en son zamana bakıp 1 gün arttırıyorum.
"""
df["InvoiceDate"].max()

In [ ]:
"""
Veri setimizi ayrı bir dataframe e atıp yazının başında bahsettiğim değişkenlere dönüştürüyoruz.
"""
today_date = dt.datetime(2011, 12, 10)
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'Invoice': lambda num: len(num),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

rfm.columns = ['Recency', 'Frequency', 'Monetary'] # Kolon isimlerini düzenliyoruz

rfm = rfm[(rfm["Monetary"]) > 0 & (rfm["Frequency"] > 0)]

In [ ]:
"""
Bu kısımda calculation yapıyoruz. Predict yapmıyoruz. O yüzden pandasın qcut fonk. ile eşit parçalara bölüp RFM skorlamasını yapıyoruz.
"""
rfm["RecencyScore"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1]) # Diğerlerine bakacak olursak bunun ters label olarak verilmesinin sebebi en düşük değerinin en değerli olmasıdır.

rfm["FrequencyScore"] = pd.qcut(rfm['Frequency'], 5, labels=[1, 2, 3, 4, 5])

rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

rfm["RFM_SCORE"] = (rfm['RecencyScore'].astype(str) +
                    rfm['FrequencyScore'].astype(str) +
                    rfm['MonetaryScore'].astype(str))


rfm["RFM_SCORE"]= rfm["RFM_SCORE"].astype(int) ## ileride segmentlerin ortalama RFM değerlerini görüntülemek için tür dönüşümü yapıyoruz

In [ ]:
"""
RFM skorlarına bakarak ilk 5 satırı görüntülüyoruz. Veriyi tanımak adına.
"""
rfm[rfm["RFM_SCORE"] == 555].head()

In [ ]:
rfm[rfm["RFM_SCORE"] == 111].head()

In [ ]:
"""
Segment lere ayırmamış gerek bu skorları. O yüzden bir regex kullanarak bir dict oluşturuyoruz.
"""
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At_Risk',
    r'[1-2]5': 'Cant_Loose',
    r'3[1-2]': 'About_to_Sleep',
    r'33': 'Need_Attention',
    r'[3-4][4-5]': 'Loyal_Customers',
    r'41': 'Promising',
    r'51': 'New_Customers',
    r'[4-5][2-3]': 'Potential_Loyalists',
    r'5[4-5]': 'Champions'
}

In [ ]:
"""
Oluşturuduğumuz dict ile replace ediyoruz segmentimizi.
"""
rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)

rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)

In [ ]:
"""
Segment lerin ortalama ve toplam değerlerini görüntülüyoruz.
"""
rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count"])

In [ ]:
rfm_map = rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count"]).copy()

In [ ]:
"""
Görüntülediğimiz değeri rfm_map e aktardıktan sonra infoya baktığımızda kolonlar pivot table olarak gözüküyor. Biz bunların sadece ortalamalarını görüntülemek istiyoruz.
xs fonk. ile sadece mean değerlerini alıyoruz. Ardından map ile bu kolonların üst level bilgileri ile isimlerini birleştirip yeni kolon isimleri yapıyoruz.
"""
rfm_map.info()

In [ ]:
rfm_map=rfm_map.xs('mean',axis=1, level=1, drop_level=False)
rfm_map.columns = rfm_map.columns.map('_'.join)

In [ ]:
rfm_map

In [ ]:
"""
Burada segmentleri stack bar plot ile inceliyoruz. Böylelikle segmenler nasıl atanmış daha iyi anlıyoruz.
"""
rfm_map.plot(kind="bar",stacked=True,figsize=(14,10))
plt.legend(loc="lower left",bbox_to_anchor=(0.8,1.0))
plt.show()


Örnek 3 adet  segment incelemesi


In [ ]:
rfm[rfm["Segment"] == "About_to_Sleep"].groupby("Segment").agg(["mean"])

In [ ]:
rfm[rfm["Segment"] == "About_to_Sleep"].describe().T


In [ ]:
rfm[rfm["Segment"] == "At_Risk"].groupby("Segment").agg(["mean"])

In [ ]:
rfm[rfm["Segment"] == "At_Risk"].describe().T

In [ ]:
rfm[rfm["Segment"] == "New_Customers"].groupby("Segment").agg(["mean"])

In [ ]:
rfm[rfm["Segment"] == "New_Customers"].describe().T

# KMeans ile kümeleme

In [ ]:
rfm_cluster = rfm.iloc[:,0:3].copy()
rfm_cluster.info()

In [ ]:
from sklearn.cluster import KMeans
Nc = range(1, 21)
kmeans = [KMeans(n_clusters=i) for i in Nc]
score = [kmeans[i].fit(rfm_cluster).score(rfm_cluster) for i in range(len(kmeans))]
sns.pointplot(x=list(Nc),y=score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=1231).fit(rfm_cluster)

In [ ]:
rfm_cluster['cluster'] = kmeans.labels_

Küme 0 yüksek R değerine sahiptir ve bu kötü bir şeydir. Bu yüzden en düşük sınıf 0 dır.

* Cluster 0 
* Cluster 1  
* Cluster 2 
* Cluster 3 
* Cluster 4 

5 sınıf olarak değerlendirebiliriz. Ancak KMeans n_clusters kısmını değiştirilirse küme sayısı değişecektir.
Pointplottan yararlanarak en iyi küme sayısının 5 olacağına karar verdim. Bunun gibi grafiklere Elbow Curve deniyor. Grafiği okuyarak en iyi değere karar veriliyor.

Alt satırlarda kümelerin daha iyi anlaşılması için grafikler çizilmiştir.


In [ ]:
rfm_cluster[rfm_cluster.cluster == 0].head(10)

In [ ]:
sns.boxplot(rfm_cluster.cluster,rfm_cluster.Recency)
plt.show()

In [ ]:
sns.boxplot(rfm_cluster.cluster,rfm_cluster.Frequency)
plt.show()

In [ ]:
sns.boxplot(rfm_cluster.cluster,rfm_cluster.Monetary)
plt.show()

In [ ]:
rfm_comp = rfm_cluster.merge(rfm,left_on=['Customer ID','Recency','Frequency','Monetary'],right_on=['Customer ID','Recency','Frequency','Monetary'])

# Drop etmekten kurtulmak için r,f,m yi ekledik yoksa gereksizdi

In [ ]:
rfm_comp.head(10)

In [ ]:
rfm_comp.groupby(["Segment",'cluster']).agg(['min','mean','max'])